In [1]:
import json
import sqlite3
from collections import Counter

import pandas as pd

In [2]:
%%bash
# remove test.db and ignore the error if it does not exist
rm test.db | exit 0

In [3]:
DATA_ROOT = "../exascale_data"
con = sqlite3.connect("test.db")

In [4]:
clusters_table = """CREATE TABLE IF NOT EXISTS clusters (
    "index" INTEGER,
    "cluster_id" TEXT,
    "cluster_prefix" TEXT,
    "node_ids" TEXT
)"""

edges_table = """CREATE TABLE IF NOT EXISTS edges (
    "index" INTEGER,
    "node1" TEXT,
    "node2" TEXT,
    "score" REAL,
    "edge_type" TEXT,
    "directed" INTEGER
)"""

nodes_raw_table = """CREATE TABLE IF NOT EXISTS nodes_raw (
    "index" INTEGER,
    "node_id" TEXT,
    "node_type" TEXT,
    "gene_model_type" TEXT,
    "gene_symbol" TEXT,
    "go_description" TEXT,
    "go_terms" TEXT,
    "mapman_bin" TEXT,
    "mapman_description" TEXT,
    "mapman_name" TEXT,
    "pheno_aragwas_id" TEXT,
    "pheno_description" TEXT,
    "pheno_pto_description" TEXT,
    "pheno_pto_name" TEXT,
    "pheno_reference" TEXT,
    "tair_computational_description" TEXT,
    "tair_curator_summary" TEXT,
    "tair_short_description" TEXT,
    "transcript" TEXT
)"""

nodes_table = """CREATE TABLE IF NOT EXISTS nodes (
    "mindex" INTEGER,
    "node_id" TEXT,
    "node_type" TEXT,
    "gene_model_type" TEXT,
    "gene_symbol" TEXT,
    "pheno_aragwas_id" TEXT,
    "pheno_description" TEXT,
    "pheno_pto_description" TEXT,
    "pheno_pto_name" TEXT,
    "pheno_reference" TEXT,
    "transcripts" TEXT,
    "go_terms" TEXT,
    "go_infos" TEXT,
    "mapman_infos" TEXT,
    "tair_infos" TEXT
)"""

cur_clusters = con.execute(clusters_table)
cur_edges = con.execute(edges_table)
cur_nodes_raw = con.execute(nodes_raw_table)
cur_nodes = con.execute(nodes_table)

assert cur_clusters.fetchall() == []
assert cur_edges.fetchall() == []
assert cur_nodes_raw.fetchall() == []
assert cur_nodes.fetchall() == []

columns_nodes_raw = list(pd.read_sql("""SELECT * FROM nodes_raw""", con).columns)
columns_nodes_raw

['index',
 'node_id',
 'node_type',
 'gene_model_type',
 'gene_symbol',
 'go_description',
 'go_terms',
 'mapman_bin',
 'mapman_description',
 'mapman_name',
 'pheno_aragwas_id',
 'pheno_description',
 'pheno_pto_description',
 'pheno_pto_name',
 'pheno_reference',
 'tair_computational_description',
 'tair_curator_summary',
 'tair_short_description',
 'transcript']

In [5]:
df1 = pd.read_csv(f"{DATA_ROOT}/prerelease/edge_data/mentha_AT-PPI-3702_040319.to-kbase-spec.tsv", sep="\t")
df1.head()

,node1,node2,score,edge_type,directed
0,AT2G18790,AT1G09530,1.000,protein-protein-interaction_Mentha_A_thaliana_...,1
1,AT4G33430,AT4G39400,1.000,protein-protein-interaction_Mentha_A_thaliana_...,1
2,AT3G20740,AT1G02580,0.999,protein-protein-interaction_Mentha_A_thaliana_...,1
3,AT5G35410,AT4G33000,0.999,protein-protein-interaction_Mentha_A_thaliana_...,1
4,AT3G62980,AT3G23050,0.999,protein-protein-interaction_Mentha_A_thaliana_...,1


In [6]:
df0 = pd.read_csv(f"{DATA_ROOT}/prerelease/edge_data/ATRM_TF_to_Target_LitCurated_01082020.to-kbase-spec.tsv", sep="\t")
df0.head()

,node1,node2,score,edge_type
0,AT5G67420,AT4G09820,1,transcription-factor-regulatory-interaction_li...
1,AT2G20180,AT4G23750,1,transcription-factor-regulatory-interaction_li...
2,AT2G20180,AT1G80340,1,transcription-factor-regulatory-interaction_li...
3,AT2G20180,AT2G29090,1,transcription-factor-regulatory-interaction_li...
4,AT2G20180,AT2G40220,1,transcription-factor-regulatory-interaction_li...


In [7]:
df0.to_sql("edges", con, if_exists="append")

In [8]:
df1.to_sql("edges", con, if_exists="append")

In [9]:
df2 = pd.DataFrame({'name' : ['User 4', 'User 5', 'User 6']})
df3 = pd.DataFrame(dict(letter=["a"]*len(df2)))
pd.concat([df2, df3], axis=1)

,name,letter
0,User 4,a
1,User 5,a
2,User 6,a


In [10]:
df5 = pd.read_csv(
    f"{DATA_ROOT}/prerelease/cluster_data/out.aranetv2_subnet_AT-CX_top10percent_anno_AF_082919.abc.I2_named.tsv",
    sep="\t"
)
df6 = pd.DataFrame(dict(cluster_prefix=["markov_i2"]*len(df5))) # i2 - inflation of 2
df7 = pd.concat([df5, df6], axis=1)
df7.tail()

,cluster_id,node_ids,cluster_prefix
3097,Cluster3098,"AT2G20080,AT2G20660",markov_i2
3098,Cluster3099,"AT3G62280,AT5G23210",markov_i2
3099,Cluster3100,"AT1G75890,AT3G15630",markov_i2
3100,Cluster3101,"AT1G67670,AT1G78710",markov_i2
3101,Cluster3102,"AT1G54870,AT5G54740",markov_i2


In [11]:
pd.read_sql("""select * from clusters order by "index" desc limit 5""", con)

,index,cluster_id,cluster_prefix,node_ids


In [12]:
df7.to_sql("clusters", con, if_exists="append")

In [13]:
df8 = pd.read_csv(
    f"{DATA_ROOT}/prerelease/cluster_data/out.aranetv2_subnet_AT-CX_top10percent_anno_AF_082919.abc.I4_named.tsv",
    sep="\t"
)
df9 = pd.DataFrame(dict(cluster_prefix=["markov_i4"]*len(df8))) # i4 - inflation of 4
dfx = pd.concat([df8, df9], axis=1)
dfx.tail()

,cluster_id,node_ids,cluster_prefix
3688,Cluster3689,"AT2G20080,AT2G20660",markov_i4
3689,Cluster3690,"AT3G62280,AT5G23210",markov_i4
3690,Cluster3691,"AT1G75890,AT3G15630",markov_i4
3691,Cluster3692,"AT1G67670,AT1G78710",markov_i4
3692,Cluster3693,"AT1G54870,AT5G54740",markov_i4


In [14]:
dfx.to_sql("clusters", con, if_exists="append")

In [15]:
pd.read_sql("""select * from clusters order by "index" desc limit 5""", con)

,index,cluster_id,cluster_prefix,node_ids
0,3692,Cluster3693,markov_i4,"AT1G54870,AT5G54740"
1,3691,Cluster3692,markov_i4,"AT1G67670,AT1G78710"
2,3690,Cluster3691,markov_i4,"AT1G75890,AT3G15630"
3,3689,Cluster3690,markov_i4,"AT3G62280,AT5G23210"
4,3688,Cluster3689,markov_i4,"AT2G20080,AT2G20660"


In [16]:
# columns for each node source file
!find {DATA_ROOT}/prerelease/node_tables -maxdepth 1 -mindepth 1 | xargs head -n 1

==> ../exascale_data/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_gene-GOslimTAIR-v010121.tsv <==
node_id	go_description	go_terms	transcript	node_type

==> ../exascale_data/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_pheno-AraPheno-v062716.tsv <==
node_id	node_type	pheno_aragwas_id	pheno_description	pheno_pto_name	pheno_pto_description	pheno_reference

==> ../exascale_data/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_gene-MapMan-vX4.2.tsv <==
mapman_bin	mapman_name	transcript	mapman_description	node_id	node_type

==> ../exascale_data/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_gene-TAIR-v2018.tsv <==
node_id	transcript	gene_model_type	tair_short_description	tair_curator_summary	tair_computational_description	node_type


In [17]:
dfn0 = pd.read_csv(
    f"{DATA_ROOT}/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_gene-MapMan-vX4.2.tsv",
    sep="\t"
)
dfn1 = pd.read_csv(
    f"{DATA_ROOT}/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_gene-TAIR-v2018.tsv",
    sep="\t"
)
dfn2 = pd.read_csv(
    f"{DATA_ROOT}/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_pheno-AraPheno-v062716.tsv",
    sep="\t"
)
dfn3 = pd.read_csv(
    f"{DATA_ROOT}/prerelease/node_tables/20210107_ExaKBase_AT_nodetab_gene-GOslimTAIR-v010121.tsv",
    sep="\t"
)
merge_df = pd.read_csv(
    f"{DATA_ROOT}/prerelease/aranet2-aragwas-MERGED-AMW-v2_091319_nodeTable.csv",
    usecols=lambda col: col in columns_nodes_raw
)
dfn0.to_sql("nodes_raw", con, if_exists="append")
dfn1.to_sql("nodes_raw", con, if_exists="append")
dfn2.to_sql("nodes_raw", con, if_exists="append")
dfn3.to_sql("nodes_raw", con, if_exists="append")
merge_df.to_sql("nodes_raw", con, if_exists="append")

In [18]:
# How many rows per node are there? If counts > 1 then some sort of merging is required.
pd.read_sql("""
SELECT node_id, node_type, COUNT(*) as counts
    FROM nodes_raw
    GROUP BY node_id, node_type
    ORDER BY count(*) DESC
    LIMIT 5
""", con)

,node_id,node_type,counts
0,AT1G07890,gene,92
1,AT4G30820,gene,90
2,AT1G20620,gene,74
3,AT4G20260,gene,66
4,AT1G51270,gene,61


In [19]:
# if counts <= 1 for each node_id then any represenative will do
pd.concat([pd.read_sql(f"""
SELECT '{key}' as key,
        COUNT(DISTINCT({key})) as counts,
        node_id
    FROM nodes_raw
    GROUP BY node_id
    ORDER BY COUNT(DISTINCT({key})) DESC
""", con).head(1) for key in columns_nodes_raw[1:]])

,key,counts,node_id
0,node_id,1,SDV
0,node_type,1,SDV
0,gene_model_type,1,ATMG01400
0,gene_symbol,1,ATMG01400
0,go_description,53,AT4G08920
0,go_terms,52,AT4G08920
0,mapman_bin,5,AT3G17240
0,mapman_description,54,AT4G30820
0,mapman_name,5,AT3G17240
0,pheno_aragwas_id,1,SDV


In [20]:
pd.concat([pd.read_sql(f"""
SELECT '{key}' as key,
        COUNT(DISTINCT({key})) as counts,
        transcript
    FROM nodes_raw
    WHERE transcript IS NOT NULL
    GROUP BY transcript
    ORDER BY COUNT(DISTINCT({key})) DESC
""", con).head(1) for key in columns_nodes_raw[1:]])

,key,counts,transcript
0,node_id,1,ATMG01400.1
0,node_type,1,ATMG01400.1
0,gene_model_type,1,ATMG01400.1
0,gene_symbol,1,ATMG01400.1
0,go_description,15,AT1G35720.1
0,go_terms,14,AT1G35720.1
0,mapman_bin,4,AT3G19710.1
0,mapman_description,4,AT3G17240.3
0,mapman_name,4,AT3G19710.1
0,pheno_aragwas_id,0,ATMG01400.1


In [21]:
pd.concat([pd.read_sql(f"""
SELECT '{key}' as key,
        COUNT(DISTINCT({key})) as counts,
        mapman_bin
    FROM nodes_raw
    WHERE mapman_bin IS NOT NULL
    GROUP BY mapman_bin
    ORDER BY COUNT(DISTINCT({key})) DESC
""", con).head(1) for key in ("mapman_bin", "mapman_description", "mapman_name")])

,key,counts,mapman_bin
0,mapman_bin,1,9.5.2.5
0,mapman_description,9813,35.1
0,mapman_name,1,9.5.2.5


In [22]:
pd.read_sql(f"""
SELECT mapman_bin, COUNT(DISTINCT(mapman_description)) as counts
    FROM nodes_raw
    WHERE True
        AND mapman_bin IS NOT NULL
    GROUP BY mapman_bin
    ORDER BY COUNT(DISTINCT(mapman_description)) DESC
    LIMIT 100
""", con)

,mapman_bin,counts
0,35.1,9813
1,35.2,9261
2,15.5.32,363
3,50.1.13,354
4,50.2.7,293
...,...,...
95,15.5.3.5,35
96,15.5.1.2,35
97,13.7.2.1,35
98,50.3.6,34


In [23]:
# go_terms has a many to one relationship with node_id
go_terms_raw = tuple(frozenset(pd.read_sql(f"""
SELECT node_id, go_terms
    FROM nodes_raw
    WHERE node_id = 'AT4G08920'
""", con)["go_terms"]))
len(go_terms_raw), go_terms_raw[:5]

(53, ('GO:0010343', 'GO:0043153', 'GO:2000652', 'GO:0009646', 'GO:0060918'))

In [24]:
# go_description has a many to one relationship with node_id
go_description_raw = tuple(frozenset(pd.read_sql(f"""
SELECT node_id, go_description
    FROM nodes_raw
    WHERE node_id = 'AT4G08920'
""", con)["go_description"]))
len(go_description_raw), go_description_raw[:5]

(54,
 ('protein binding',
  'flavin adenine dinucleotide metabolic process',
  'regulation of meristem growth',
  'DNA binding',
  'nucleus'))

In [25]:
# go_terms has a many to one relationship with transcript
go_terms_trsx_raw = tuple(frozenset(pd.read_sql(f"""
SELECT transcript, go_terms
    FROM nodes_raw
    WHERE transcript = 'AT1G26440.2'
""", con)["go_terms"]))
len(go_terms_trsx_raw), go_terms_trsx_raw[:5]

(15, ('GO:0071705', 'GO:0099503', 'GO:0015210', 'GO:0009504', 'GO:0005783'))

In [26]:
# mapman_bin has a many to one relationship with node_id
mapman_bin_raw = tuple(frozenset(pd.read_sql(f"""
SELECT node_id, mapman_bin
    FROM nodes_raw
    WHERE node_id = 'AT3G17240'
""", con)["mapman_bin"]))
len(mapman_bin_raw), mapman_bin_raw[:5]

(6, ('2.3.4.3', '4.2.7.2.3', None, '35.1', '2.2.1.3'))

In [27]:
# mapman_bin has a many to one relationship with transcript
mapman_bin_trsx_raw = tuple(frozenset(pd.read_sql(f"""
SELECT node_id, mapman_bin
    FROM nodes_raw
    WHERE transcript = 'AT1G18040.1'
""", con)["mapman_bin"]))
len(mapman_bin_trsx_raw), mapman_bin_trsx_raw[:5]

(5, ('13.1.1.2.4', '18.4.3.1.4', '15.3.5.6.2.2', None, '14.6.3.1.3'))

In [28]:
# pheno_reference differs only in the URL, so keep the URL version
pheno_reference_node_ids = '", "'.join(list(pd.read_sql("""
SELECT node_id, pheno_reference
    FROM nodes_raw
    GROUP BY node_id
    HAVING COUNT(DISTINCT(pheno_reference)) = 2
    ORDER BY COUNT(DISTINCT(pheno_reference)) DESC
""", con)["node_id"]))
pd.read_sql(f"""
SELECT node_id, MAX(pheno_reference), MIN(pheno_reference)
    FROM nodes_raw
    WHERE node_id in ("{pheno_reference_node_ids}")
    GROUP BY node_id
""", con)

,node_id,MAX(pheno_reference),MIN(pheno_reference)
0,As2,https://doi.org/10.1038/nature08800,doi:10.1038/nature08800
1,As75,https://doi.org/10.1038/nature08800,doi:10.1038/nature08800
2,At2,https://doi.org/10.1038/nature08800,doi:10.1038/nature08800
3,Co59,https://doi.org/10.1038/nature08800,doi:10.1038/nature08800
4,Mo98,https://doi.org/10.1038/nature08800,doi:10.1038/nature08800
5,Na23,https://doi.org/10.1038/nature08800,doi:10.1038/nature08800
6,SDV,https://doi.org/10.1038/nature08800,doi:10.1038/nature08800


In [29]:
concat_query = """
SELECT
        MIN("index") as mindex,
        node_id,
        node_type,
        gene_model_type,
        gene_symbol,
        pheno_aragwas_id,
        pheno_description,
        pheno_pto_description,
        pheno_pto_name,
        MAX(pheno_reference),
        '['
        || group_concat(
            CASE WHEN length(transcript)>0
            THEN printf('"%s"', transcript)
            ELSE '""' END
        )
        || ']' as transcripts,
        '['
        || group_concat(
            CASE WHEN length(go_terms)>0
            THEN printf('"%s"', go_terms)
            ELSE '""' END
        )
        || ']' as go_terms,
        '['
        || group_concat(
            CASE WHEN length(go_terms)>0
            THEN printf(
                '{"transcript": "%s", "go_term": "%s", "go_description": "%s"}',
                transcript, go_terms, go_description
            )
            ELSE '{}' END
        )
        || ']' as go_infos,
        '['
        || group_concat(
            CASE WHEN length(mapman_bin)>0
            THEN printf(
                '{"transcript": "%s", "bin": "%s", "name": "%s", "description": "%s"}',
                transcript, mapman_bin, mapman_name, mapman_description
            )
            ELSE '{}' END
        )
        || ']' as mapman_infos,
        '['
        || group_concat(
            CASE WHEN length(tair_short_description)>0
            THEN printf(
                '{
                    "transcript": "%s",
                    "gene_model_type": "%s",
                    "tair_short_description": "%s",
                    "tair_curator_summary": "%s",
                    "tair_computational_description": "%s"
                }',
                transcript,
                gene_model_type,
                tair_short_description,
                tair_curator_summary,
                tair_computational_description
            )
            ELSE '{}' END
        )
        || ']' as tair_infos
    FROM nodes_raw
    GROUP BY node_id, node_type
    ORDER BY count(*) DESC
"""

In [30]:
cur_catted = con.execute(f"""INSERT INTO nodes {concat_query}""")
cur_catted.fetchall()

[]

In [31]:
catted = pd.read_sql("""SELECT * from nodes""", con)
catted.tail(10)

,mindex,node_id,node_type,gene_model_type,gene_symbol,pheno_aragwas_id,pheno_description,pheno_pto_description,pheno_pto_name,pheno_reference,transcripts,go_terms,go_infos,mapman_infos,tair_infos
24604,20700,AT5G49090,gene,transposable_element_gene,None,None,None,None,None,None,"[""AT5G49090.1""]","[""""]",[{}],[{}],"[{\n ""transcript"": ""AT5G490..."
24605,19958,AT5G49380,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24606,20062,AT5G50645,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24607,20326,AT5G53588,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24608,20620,AT5G57181,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24609,20633,AT5G57310,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24610,20669,AT5G57730,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24611,20919,AT5G60548,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24612,20976,AT5G61228,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]
24613,21245,AT5G64342,gene,None,None,None,None,None,None,None,"[""""]","[""""]",[{}],[{}],[{}]


In [32]:
transcripts = frozenset([transcript for transcript in json.loads(catted["transcripts"].values[0]) if transcript])
len(transcripts), tuple(transcripts)[:2]

(8, ('AT1G07890.8', 'AT1G07890.1'))

In [33]:
# note that the mapman data is tied to the transcript.
[mapman for mapman in json.loads(catted["mapman_infos"].values[0]) if mapman != {}][:2]

[{'transcript': 'AT1G07890.5',
  'bin': '10.4.1.1',
  'name': '.Redox homeostasis.hydrogen peroxide removal.ascorbate-glutathione cycle.ascorbate peroxidase (APX)',
  'description': 'ascorbate peroxidase (APX) (original description: pep chromosome:TAIR10:1:2437478:2439665:1 gene:AT1G07890 transcript:AT1G07890.5 gene_biotype:protein_coding transcript_biotype:protein_coding gene_symbol:APX1 description:MEE6 [Source:UniProtKB/TrEMBL;Acc:A0A178W5I1])'},
 {'transcript': 'AT1G07890.3',
  'bin': '10.4.1.1',
  'name': '.Redox homeostasis.hydrogen peroxide removal.ascorbate-glutathione cycle.ascorbate peroxidase (APX)',
  'description': 'ascorbate peroxidase (APX) (original description: pep chromosome:TAIR10:1:2437725:2439665:1 gene:AT1G07890 transcript:AT1G07890.3 gene_biotype:protein_coding transcript_biotype:protein_coding gene_symbol:APX1 description:MEE6 [Source:UniProtKB/TrEMBL;Acc:A0A178W5I1])'}]

In [34]:
# note that the go term data is tied to the transcript.
# there appears to be a one to many relationship from transcripts to GO terms
go_infos_json = [json.dumps(go_info) for go_info in json.loads(catted["go_infos"].values[0]) if go_info != {}]
assert len(go_infos_json) == len(frozenset(go_infos_json))

In [35]:
# the list of go terms per gene is also useful to have available
go_terms = tuple(frozenset([go_term for go_term in json.loads(catted["go_terms"].values[0]) if go_term != ""]))
go_terms[:5]

('GO:0009699', 'GO:0005576', 'GO:0009809', 'GO:0005737', 'GO:0016688')

In [36]:
# note that the tair data is also tied to the transcript.
# there appears to be a one to one relationship from transcript to tair info
tair_infos_json = frozenset([json.dumps(tair_info) for tair_info in json.loads(catted["tair_infos"].values[0]) if tair_info != {}])
len(tair_infos_json), len(transcripts), tuple(frozenset(tair_infos_json))[:2]


(8,
 8,
 ('{"transcript": "AT1G07890.6", "gene_model_type": "protein_coding", "tair_short_description": "ascorbate peroxidase 1", "tair_curator_summary": "Encodes a cytosolic ascorbate peroxidase APX1.  Ascorbate peroxidases are enzymes that scavenge hydrogen peroxide in plant cells.  Eight types of APX have been described for Arabidopsis: three cytosolic (APX1, APX2, APX6), two chloroplastic types (stromal sAPX, thylakoid tAPX), and three microsomal (APX3, APX4, APX5) isoforms. At least part of the induction of heat shock proteins during light stress in Arabidopsis is mediated by H2O2 that is scavenged by APX1. Expression of the gene is downregulated in the presence of paraquat, an inducer of photoxidative stress. The mRNA is cell-to-cell mobile.", "tair_computational_description": "ascorbate peroxidase 1;(source:Araport11)"}',
  '{"transcript": "AT1G07890.7", "gene_model_type": "protein_coding", "tair_short_description": "ascorbate peroxidase 1", "tair_curator_summary": "Encodes a cy

***
Did this actually solve the problem? That is, `ATCG00690` is not in the merged nodes table csv.

In [37]:
problem_gene = pd.read_sql("""SELECT * from nodes where node_id="ATCG00690" """, con)
problem_gene

,mindex,node_id,node_type,gene_model_type,gene_symbol,pheno_aragwas_id,pheno_description,pheno_pto_description,pheno_pto_name,pheno_reference,transcripts,go_terms,go_infos,mapman_infos,tair_infos
0,34,ATCG00690,gene,None,None,None,None,None,None,None,"[""ATCG00690.1"",""ATCG00690.1"",""ATCG00690.1"","""",...","["""","""",""GO:0009535"",""GO:0009539"",""GO:0016168"",...","[{},{},{""transcript"": ""ATCG00690.1"", ""go_term""...","[{""transcript"": ""ATCG00690.1"", ""bin"": ""1.1.1.2...","[{},{\n ""transcript"": ""ATCG..."


Yes. Yes it did. *phew*

***
Below `node_id` is used, but in these cases it always represents a gene

In [38]:
inspect_query = """SELECT
        MIN("index") as mindex,
        node_id,
        node_type,
        '['
        || group_concat(
            CASE WHEN length(tair_computational_description)>0
            THEN printf('"%s"', tair_computational_description)
            ELSE '""' END
        )
        || ']' as tair_computational_description,
        '['
        || group_concat(
            CASE WHEN length(tair_curator_summary)>0
            THEN printf('"%s"', tair_curator_summary)
            ELSE '""' END
        )
        || ']' as tair_curator_summary,
        '['
        || group_concat(
            CASE WHEN length(tair_short_description)>0
            THEN printf('"%s"', tair_short_description)
            ELSE '""' END
        )
        || ']' as tair_short_description
    FROM nodes_raw
    GROUP BY node_id, node_type
    ORDER BY count(*) DESC
"""
inspect = pd.read_sql(inspect_query, con)
inspect

,mindex,node_id,node_type,tair_computational_description,tair_curator_summary,tair_short_description
0,588,AT1G07890,gene,"["""","""","""","""","""","""","""","""",""ascorbate peroxidase...","["""","""","""","""","""","""","""","""",""Encodes a cytosolic ...","["""","""","""","""","""","""","""","""",""ascorbate peroxidase..."
1,6672,AT4G30820,gene,"["""","""","""","""","""","""","""","""","""","""","""","""","""","""","""",...","["""","""","""","""","""","""","""","""","""","""","""","""","""","""","""",...","["""","""","""","""","""","""","""","""","""","""","""","""","""","""","""",..."
2,1662,AT1G20620,gene,"["""","""","""","""","""","""",""catalase 3;(source:Araport...","["""","""","""","""","""","""",""Catalase, catalyzes the br...","["""","""","""","""","""","""",""catalase 3"",""catalase 3"",""..."
3,11000,AT4G20260,gene,"["""","""","""","""","""","""","""","""","""","""",""plasma-membran...","["""","""","""","""","""","""","""","""","""","""",""Encodes a Ca2+...","["""","""","""","""","""","""","""","""","""","""",""plasma-membran..."
4,178,AT1G51270,gene,"["""","""","""","""","""","""","""","""","""","""","""","""","""","""","""",...","["""","""","""","""","""","""","""","""","""","""","""","""","""","""","""",...","["""","""","""","""","""","""","""","""","""","""","""","""","""","""","""",..."
...,...,...,...,...,...,...
24609,20633,AT5G57310,gene,"[""""]","[""""]","[""""]"
24610,20669,AT5G57730,gene,"[""""]","[""""]","[""""]"
24611,20919,AT5G60548,gene,"[""""]","[""""]","[""""]"
24612,20976,AT5G61228,gene,"[""""]","[""""]","[""""]"


In [39]:
# is it the case that there is only one nonempty tair_computational_description for a given node_id?
distinct_tair_computational_descriptions = tuple(frozenset([tcd for tcd in json.loads(inspect["tair_computational_description"].values[0]) if tcd != ""]))
assert len(distinct_tair_computational_descriptions) == 1
distinct_tair_computational_descriptions[0]

'ascorbate peroxidase 1;(source:Araport11)'

In [40]:
# is it the case that there is only one nonempty tair_short_description for a given node_id?
distinct_tair_short_descriptions = tuple(frozenset([tcs for tcs in json.loads(inspect["tair_short_description"].values[0]) if tcs != ""]))
assert len(distinct_tair_short_descriptions) == 1
distinct_tair_short_descriptions[0]

'ascorbate peroxidase 1'

In [41]:
# is it the case that there is only one nonempty tair_curator_summary for a given node_id?
distinct_tair_curator_summarys = tuple(frozenset([tcs for tcs in json.loads(inspect["tair_curator_summary"].values[0]) if tcs != ""]))
assert len(distinct_tair_curator_summarys) == 1
distinct_tair_curator_summarys[0]

'Encodes a cytosolic ascorbate peroxidase APX1.  Ascorbate peroxidases are enzymes that scavenge hydrogen peroxide in plant cells.  Eight types of APX have been described for Arabidopsis: three cytosolic (APX1, APX2, APX6), two chloroplastic types (stromal sAPX, thylakoid tAPX), and three microsomal (APX3, APX4, APX5) isoforms. At least part of the induction of heat shock proteins during light stress in Arabidopsis is mediated by H2O2 that is scavenged by APX1. Expression of the gene is downregulated in the presence of paraquat, an inducer of photoxidative stress. The mRNA is cell-to-cell mobile.'

In [42]:
# no, that is not the case
pd.read_sql("""
SELECT node_id,
        COUNT(DISTINCT(tair_curator_summary)) as counts
    FROM nodes_raw
    GROUP BY node_id
    ORDER BY COUNT(DISTINCT(tair_curator_summary)) DESC, node_id
""", con).head(1)

,node_id,counts
0,AT1G15750,3


In [43]:
# some summaries differ, although some in trivial ways, for instance
# "embryogeneis." is followed by one and two spaces below
# map null to the value '---'
valsdf = pd.read_sql("""
SELECT node_id,
        coalesce(tair_curator_summary, '---') as 'values'
    FROM nodes_raw
    WHERE node_id='AT1G15750'
    GROUP BY coalesce(tair_curator_summary, '---')
    ORDER BY node_id DESC
""", con)
tuple(valsdf["values"])

('Encodes a protein with several WD40 repeats at the C-terminus and predicted protein-protein interaction domains at the N-terminus.  Together with the TOPLESS-RELATED PROTEINS (TPRs), it is thought to be involved in transcriptional repression of root-promoting genes in the top help of the embryo during the transition stage of embryogenesis. The ability of IAA12 to repress transcription is diminished in a tpl-1 mutant background.',
 'Encodes a protein with several WD40 repeats at the C-terminus and predicted protein-protein interaction domains at the N-terminus.  Together with the TOPLESS-RELATED PROTEINS (TPRs), it is thought to be involved in transcriptional repression of root-promoting genes in the top help of the embryo during the transition stage of embryogenesis.  The ability of IAA12 to repress transcription is diminished in a tpl-1 mutant background.',
 'Encodes a protein with several WD40 repeats at the C-terminus and predicted protein-protein interaction domains at the N-term

***

In [44]:
print(f"""'{"', '".join(sorted(('AT1G18590', 'AT2G34420', 'AT5G05850', 'ATCG00280', 'ATCG00350', 'ATCG00680', 'ATCG00690')))}'""")

'AT1G18590', 'AT2G34420', 'AT5G05850', 'ATCG00280', 'ATCG00350', 'ATCG00680', 'ATCG00690'


In [45]:
node_checks = ('AT1G18590', 'AT2G34420', 'AT5G05850', 'ATCG00280', 'ATCG00350', 'ATCG00680', 'ATCG00690')
checks = pd.read_sql(f"""SELECT * from nodes where node_id in ("{'", "'.join(node_checks)}")""", con)
print(f"{len(node_checks)} checks, {len(checks.values)} results")
# assert len(node_checks) == len(checks.values)

7 checks, 5 results


In [46]:
checks

,mindex,node_id,node_type,gene_model_type,gene_symbol,pheno_aragwas_id,pheno_description,pheno_pto_description,pheno_pto_name,pheno_reference,transcripts,go_terms,go_infos,mapman_infos,tair_infos
0,18,ATCG00280,gene,protein_coding,PSBC,None,None,None,None,None,"[""ATCG00280.1"",""ATCG00280.1"",""ATCG00280.1"","""",...","["""","""",""GO:0009507"",""GO:0009539"",""GO:0003729"",...","[{},{},{""transcript"": ""ATCG00280.1"", ""go_term""...","[{""transcript"": ""ATCG00280.1"", ""bin"": ""1.1.1.2...","[{},{\n ""transcript"": ""ATCG..."
1,0,AT2G34420,gene,protein_coding,LHCB1.5,None,None,None,None,None,"[""AT2G34420.1"",""AT2G34420.1"",""AT2G34420.1"",""AT...","["""","""",""GO:0009534"",""GO:0009535"",""GO:0009535"",...","[{},{},{""transcript"": ""AT2G34420.1"", ""go_term""...","[{""transcript"": ""AT2G34420.1"", ""bin"": ""1.1.1.1...","[{},{\n ""transcript"": ""AT2G..."
2,163,ATCG00350,gene,protein_coding,PSAA,None,None,None,None,None,"[""ATCG00350.1"",""ATCG00350.1"",""ATCG00350.1"","""",...","["""","""",""GO:0009535"",""GO:0009536"",""GO:0009534"",...","[{},{},{""transcript"": ""ATCG00350.1"", ""go_term""...","[{""transcript"": ""ATCG00350.1"", ""bin"": ""1.1.4.2...","[{},{\n ""transcript"": ""ATCG..."
3,1496,AT1G18590,gene,protein_coding,SOT17,None,None,None,None,None,"[""AT1G18590.1"",""AT1G18590.1"","""","""",""AT1G18590....","["""","""",""GO:0005634"",""GO:0051923"",""GO:0005737"",...","[{},{},{""transcript"": """", ""go_term"": ""GO:00056...","[{""transcript"": ""AT1G18590.1"", ""bin"": ""9.5.1.9...","[{},{\n ""transcript"": ""AT1G..."
4,34,ATCG00690,gene,None,None,None,None,None,None,None,"[""ATCG00690.1"",""ATCG00690.1"",""ATCG00690.1"","""",...","["""","""",""GO:0009535"",""GO:0009539"",""GO:0016168"",...","[{},{},{""transcript"": ""ATCG00690.1"", ""go_term""...","[{""transcript"": ""ATCG00690.1"", ""bin"": ""1.1.1.2...","[{},{\n ""transcript"": ""ATCG..."


***
What about the merged node file?

In [47]:
merge_df.tail()

,node_id,node_type,transcript,gene_symbol,gene_model_type,go_description,pheno_description,pheno_pto_name,pheno_pto_description,pheno_reference
21671,ATMG01320,gene,ATMG01320.1,NAD2B,protein_coding,"NADH dehydrogenase (ubiquinone) activity, NADH...",NaN,NaN,NaN,NaN
21672,ATMG01350,gene,ATMG01350.1,ORF145C,protein_coding,molecular_function,NaN,NaN,NaN,NaN
21673,ATMG01360,gene,ATMG01360.1,COX1,protein_coding,cytochrome-c oxidase activity,NaN,NaN,NaN,NaN
21674,ATMG01370,gene,ATMG01370.1,ORF111D,protein_coding,molecular_function,NaN,NaN,NaN,NaN
21675,ATMG01400,gene,ATMG01400.1,ORF105B,protein_coding,molecular_function,NaN,NaN,NaN,NaN


In [48]:
Counter(merge_df["transcript"]).most_common(10)

[(nan, 85),
 ('AT1G01010.1', 1),
 ('AT1G01020.6', 1),
 ('AT1G01030.2', 1),
 ('AT1G01040.2', 1),
 ('AT1G01050.2', 1),
 ('AT1G01060.8', 1),
 ('AT1G01070.2', 1),
 ('AT1G01080.3', 1),
 ('AT1G01090.1', 1)]

In [49]:
Counter(merge_df["node_id"]).most_common(10)

[('As2', 1),
 ('As75', 1),
 ('At2', 1),
 ('Co59', 1),
 ('Mo98', 1),
 ('Na23', 1),
 ('SDV', 1),
 ('AT1G01010', 1),
 ('AT1G01020', 1),
 ('AT1G01030', 1)]

***
I originally thought the following would be useful if mapman data were "mostly" 1-1 with gene, but it looks like it is "mostly" 1-1 with transcript.

In [50]:
def merge_items(items):
    out = dict()
    for item in items:
        for key, value in item.items():
            if key not in out:
                out[key] = value
                continue
            value_old = out[key]
            if not isinstance(value_old, list):
                out[key] = [value_old, value]
                continue
            out[key].append(value)
    return out
        

In [51]:
cat_items = [{"name": "sookie", "bells": True}, {"name": "ming"}]
cat_out = {"name" : ["sookie", "ming"], "bells": True}
cat_merge = merge_items(cat_items)
assert json.dumps(cat_merge) == json.dumps(cat_out)
cat_merge

{'name': ['sookie', 'ming'], 'bells': True}